In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
# 1. Defina o caminho para a pasta com as imagens e JSONs
data_dir = 'UnityEyes_Windows/UnityEyes_Windows/imgs'

# Verifique se o diretório existe
if not os.path.exists(data_dir):
    print(f"Erro: O diretório '{data_dir}' não foi encontrado. Verifique o caminho.")
else:
    print(f"Diretório '{data_dir}' encontrado.")

    # 2. Inicialize uma lista para armazenar os dados de todos os frames
    all_data = []

    # 3. Percorra todos os arquivos na pasta
    for filename in os.listdir(data_dir):
        # Encontre apenas os arquivos JSON
        if filename.endswith('.json'):
            file_path = os.path.join(data_dir, filename)

            try:
                # Carregue o arquivo JSON
                with open(file_path, 'r') as f:
                    data = json.load(f)

                # 4. Extraia os dados relevantes
                look_vec_str = data['eye_details']['look_vec']
                # Converta a string "(x, y, z, w)" para uma tupla de floats
                look_vec_coords = tuple(map(float, look_vec_str.strip('()').split(',')))
                look_vec_x, look_vec_y, look_vec_z = look_vec_coords[0:3]

                # Tente extrair o pupil_size, se existir
                try:
                    pupil_size = float(data['eye_details']['pupil_size'])
                except (KeyError, ValueError):
                    pupil_size = np.nan # Use NaN se não for encontrado

                # 5. Crie a lógica para o rótulo de atenção
                # Vamos assumir que a atenção está no centro da tela, com um vetor de olhar ideal (0, 0, -1)
                # Calcule a distância Euclidiana entre o vetor de olhar e o vetor de atenção
                distance_to_center = np.sqrt(look_vec_x**2 + look_vec_y**2 + (look_vec_z - (-1))**2)

                # Defina um limiar para a atenção. Você pode ajustar este valor.
                attention_threshold = 0.2

                # Atribua o rótulo binário
                attention_label = 1 if distance_to_center < attention_threshold else 0

                # 6. Armazene os dados em um dicionário
                all_data.append({
                    'filename': filename,
                    'look_vec_x': look_vec_x,
                    'look_vec_y': look_vec_y,
                    'look_vec_z': look_vec_z,
                    'pupil_size': pupil_size,
                    'attention': attention_label
                })

            except (IOError, json.JSONDecodeError, KeyError) as e:
                print(f"Erro ao processar o arquivo {filename}: {e}")
                continue # Pula para o próximo arquivo em caso de erro

    # 7. Converta a lista de dicionários para um DataFrame do pandas
    gaze_df = pd.DataFrame(all_data)

    # 8. Salve o DataFrame em um arquivo CSV na raiz do seu projeto
    output_path = os.path.join('.', 'gaze_labels.csv')
    gaze_df.to_csv(output_path, index=False)
    print(f"\nArquivo '{output_path}' criado com sucesso! Contém {len(gaze_df)} registros.")

    # Exiba as primeiras linhas do novo DataFrame para verificar
    print("\nPrimeiras linhas do novo CSV:")
    print(gaze_df.head())
    
    print("\nDistribuição dos rótulos de atenção:")
    print(gaze_df['attention'].value_counts())

Diretório 'UnityEyes_Windows/UnityEyes_Windows/imgs' encontrado.

Arquivo '.\gaze_labels.csv' criado com sucesso! Contém 272879 registros.

Primeiras linhas do novo CSV:
     filename  look_vec_x  look_vec_y  look_vec_z  pupil_size  attention
0      1.json      0.6401     -0.2066     -0.7400    0.003006          0
1     10.json     -0.0762      0.4713     -0.8787   -0.153684          0
2    100.json     -0.0211     -0.5249     -0.8509   -0.457680          0
3   1000.json      0.7252      0.0969     -0.6817   -0.092931          0
4  10000.json      0.4163     -0.4684     -0.7793    0.068899          0

Distribuição dos rótulos de atenção:
attention
0    248909
1     23970
Name: count, dtype: int64
